# Project Proposal (TITLE)

## Project Proposal (INTRODUCTION)

In [1]:
library(tidyverse)
library(infer)
library(repr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
## Reading and tidying data from file ##
pokemon <- read_csv("https://raw.githubusercontent.com/Kooriryuu/Stat201Group2/main/Pokemon.csv")
colnames(pokemon) <- gsub(" ", "_", colnames(pokemon))
head(pokemon)

Rows: 800 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): Name, Type 1, Type 2
dbl (9): #, Total, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, Generation
lgl (1): Legendary

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


#,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Generation,Legendary
<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,FALSE
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,FALSE
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,FALSE
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,FALSE
4,Charmander,Fire,NA,309,39,52,43,60,50,65,1,FALSE
5,Charmeleon,Fire,NA,405,58,64,58,80,65,80,1,FALSE


In [3]:
## Wrangling Data (Removing legendaries and selecting relevant columns) ##
pokemon <- pokemon |>
    filter(!Legendary) |>
    select(Type_1, Type_2, Total)
head(pokemon)

Type_1,Type_2,Total
<chr>,<chr>,<dbl>
Grass,Poison,318
Grass,Poison,405
Grass,Poison,525
Grass,Poison,625
Fire,NA,309
Fire,NA,405


In [4]:
## Calculating sample mean for Bug and Non-Bug pokemon (sample size = 30) ##
set.seed(6969)
bug_sample <- pokemon |>
    filter(Type_1 == "Bug" | Type_2 == "Bug") |>
    sample_n(size = 30)

non_bug_sample <- pokemon |>
    filter(Type_1 != "Bug" & Type_2 != "Bug") |>
    sample_n(size = 30)

sample_means = tibble(
    bug_power = mean(bug_sample$Total),
    non_bug_power = mean(non_bug_sample$Total))

sample_means

bug_power,non_bug_power
<dbl>,<dbl>
376.3667,476.8667


In [5]:
## Visualising difference in power ##

